In [1]:
from datetime import date
import pandas as pd

pd.set_option('display.max_columns', None)

df = catalog.load('raw/openaire/researchproduct#parquet')

                    INFO     Loading data from raw/openaire/researchproduct#parquet             ]8;id=272187;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=299336;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (ParquetDataset)...                                                                   

In [2]:
df_research_subjects = df[['id','subjects']]
df_research_subjects

,id,subjects
0,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,"[{'provenance': None, 'subject': {'scheme': 'k..."
1,dedup_wf_002::0000eb33b6be05d67799615dd117e5b2,"[{'provenance': None, 'subject': {'scheme': 'k..."
2,dedup_wf_002::00014d5f38ed4991a97d893df4cf84e7,"[{'provenance': None, 'subject': {'scheme': 'k..."
3,dedup_wf_002::0001a843d6ffe65668859ab632a1e531,"[{'provenance': None, 'subject': {'scheme': 'k..."
4,dedup_wf_002::00027faeda057626e8abbaba310695fa,"[{'provenance': None, 'subject': {'scheme': 'k..."
...,...,...
159802,pmid_dedup__::ed04e7477d247ae376d997093be8fb91,"[{'provenance': None, 'subject': {'scheme': 'k..."
159803,pmid_dedup__::f05dc5628f6f331609cc2c34a05bb411,"[{'provenance': None, 'subject': {'scheme': 'k..."
159804,pmid_dedup__::f5fc2b12e2042175f489487698896279,"[{'provenance': None, 'subject': {'scheme': 'k..."
159805,pmid_dedup__::f79f43c98e4335a8f0270347c58bd12e,"[{'provenance': None, 'subject': {'scheme': 'k..."


In [3]:
df_research_subjects = df_research_subjects.explode('subjects').reset_index(drop=True)
df_research_subjects

,id,subjects
0,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,"{'provenance': None, 'subject': {'scheme': 'ke..."
1,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,"{'provenance': None, 'subject': {'scheme': 'ke..."
2,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,"{'provenance': None, 'subject': {'scheme': 'ke..."
3,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,"{'provenance': None, 'subject': {'scheme': 'ke..."
4,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,"{'provenance': None, 'subject': {'scheme': 'ke..."
...,...,...
1241209,pmid_dedup__::f8d3cd315536e65b096442ef75eeb8f7,"{'provenance': None, 'subject': {'scheme': 'ke..."
1241210,pmid_dedup__::f8d3cd315536e65b096442ef75eeb8f7,"{'provenance': None, 'subject': {'scheme': 'ke..."
1241211,pmid_dedup__::f8d3cd315536e65b096442ef75eeb8f7,"{'provenance': None, 'subject': {'scheme': 'ke..."
1241212,pmid_dedup__::f8d3cd315536e65b096442ef75eeb8f7,"{'provenance': None, 'subject': {'scheme': 'ke..."


In [4]:
df_subjects = pd.json_normalize(df_research_subjects['subjects'])
df_research_subjects = pd.concat([df_research_subjects['id'], df_subjects],axis=1)
df_research_subjects

,id,provenance,subject.scheme,subject.value
0,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,NaN,keyword,Gestión de incidentes
1,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,NaN,keyword,CSIRT
2,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,NaN,keyword,Ciencias Informáticas
3,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,NaN,keyword,ciberseguridad
4,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,NaN,keyword,Ngen
...,...,...,...,...
1241209,pmid_dedup__::f8d3cd315536e65b096442ef75eeb8f7,NaN,keyword,health monitoring
1241210,pmid_dedup__::f8d3cd315536e65b096442ef75eeb8f7,NaN,keyword,https://purl.org/becyt/ford/4.3
1241211,pmid_dedup__::f8d3cd315536e65b096442ef75eeb8f7,NaN,keyword,Animals
1241212,pmid_dedup__::f8d3cd315536e65b096442ef75eeb8f7,NaN,keyword,https://purl.org/becyt/ford/4


## Paso 1: Convierto tipos y selecciono columnas con cardinalidad 1 con respecto a cada research product
+ info en https://graph.openaire.eu/docs/data-model/entities/research-product

In [5]:
def land_openaire_rel_researchproduct_subject(df: pd.DataFrame)-> pd.DataFrame:

    df_research_subjects = df[['id','subjects']]

    df_research_subjects = df_research_subjects.explode('subjects').reset_index(drop=True)

    df_subjects = pd.json_normalize(df_research_subjects['subjects'])
    df_research_subjects = pd.concat([df_research_subjects['id'], df_subjects],axis=1)

    return df_research_subjects


In [6]:
df_research_subjects = land_openaire_rel_researchproduct_subject(df)

In [7]:
df_research_subjects

,id,provenance,subject.scheme,subject.value
0,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,NaN,keyword,Gestión de incidentes
1,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,NaN,keyword,CSIRT
2,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,NaN,keyword,Ciencias Informáticas
3,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,NaN,keyword,ciberseguridad
4,dedup_wf_002::00004d08998bcb6f35ce5d53811f12e9,NaN,keyword,Ngen
...,...,...,...,...
1241209,pmid_dedup__::f8d3cd315536e65b096442ef75eeb8f7,NaN,keyword,health monitoring
1241210,pmid_dedup__::f8d3cd315536e65b096442ef75eeb8f7,NaN,keyword,https://purl.org/becyt/ford/4.3
1241211,pmid_dedup__::f8d3cd315536e65b096442ef75eeb8f7,NaN,keyword,Animals
1241212,pmid_dedup__::f8d3cd315536e65b096442ef75eeb8f7,NaN,keyword,https://purl.org/becyt/ford/4
